In [29]:
import yfinance as yf
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from arch import arch_model
import matplotlib.pyplot as plt
from scipy.stats import t

In [30]:
stocklist = ['JNJ', 'BTC-USD', 'AAPL']
mean_models = ['Constant', 'AR']
vols = ["GARCH", "EGARCH"]
dists = ['normal', 't']

results = []


In [31]:
def quantile_VaR(returns, confidence_level=0.95):
    """
    Calculate Value at Risk (VaR) using the quantile method.

    Parameters:
        returns (array-like): Historical returns (losses should be negative).
        confidence_level (float): Confidence level (e.g., 0.95 for 95% confidence).

    Returns:
        float: VaR (expressed as a positive number representing the potential loss).
    """
    # Compute the quantile corresponding to the tail probability.
    # For a 95% confidence level, we take the 5th percentile.
    quantile_value = np.quantile(returns, 1 - confidence_level)

    # Since losses are negative, the VaR is the absolute value of this quantile.
    VaR = -quantile_value
    return VaR

In [ ]:
confidence_level = 0.95

for stock in stocklist:
    dprice = yf.download(stock, start="2015-01-01", end="2025-01-01", auto_adjust=True, interval='1wk')
    dprice['log_return'] = 100 * np.log(dprice['Close'] / dprice['Close'].shift(1))
    logrets = dprice['log_return'].dropna()
    for mean in mean_models:
        for dist in dists:
            for vol in vols:
                garch_model = arch_model(logrets, mean=mean, vol=vol, lags=1 if mean=="AR" else 0, p=1, q=1, dist=dist)
                garch_result = garch_model.fit(disp='off')
                garch_forecast = garch_result.forecast(horizon=1)

                mean_forecast = logrets.mean()
                sigma_forecast = np.sqrt(garch_forecast.variance.iloc[-1].values[0])

                if dist == 'normal':
                    GARCH_VaR = -mean_forecast + 1.65 * sigma_forecast
                elif dist == 't':
                    scale_param = (sigma_forecast * np.sqrt((garch_result.params['nu'] - 2) / garch_result.params['nu']))
                    GARCH_VaR = -t.ppf(1 - confidence_level, garch_result.params['nu'], mean_forecast, scale_param)

                quant_VaR = quantile_VaR(logrets, confidence_level=confidence_level)

                results.append({'stock':stock, 'mean': mean, 'dist': dist, 'vol': vol, 'meanret': mean_forecast, 'aic': garch_result.aic, 'GARCH_VaR': GARCH_VaR, 'quant_VaR': quant_VaR})

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [33]:
best_model = []

for stock in stocklist:
    best_mod = min([x for x in results if x['stock'] == stock], key=lambda x: x['aic']) # for each stock, select the best model based on AIC
    best_model.append(best_mod)

In [35]:
best_model

[{'stock': 'JNJ',
  'mean': 'AR',
  'dist': 't',
  'vol': 'GARCH',
  'meanret': np.float64(0.11340282069253148),
  'aic': 2271.0002728242193,
  'GARCH_VaR': np.float64(3.146115256432267),
  'quant_VaR': np.float64(3.2665112952789057)},
 {'stock': 'BTC-USD',
  'mean': 'AR',
  'dist': 't',
  'vol': 'EGARCH',
  'meanret': np.float64(1.107635362884434),
  'aic': 3758.3470395873665,
  'GARCH_VaR': np.float64(10.405634385255166),
  'quant_VaR': np.float64(14.892623315471734)},
 {'stock': 'AAPL',
  'mean': 'AR',
  'dist': 't',
  'vol': 'GARCH',
  'meanret': np.float64(0.45014988003368234),
  'aic': 2754.5547592172647,
  'GARCH_VaR': np.float64(4.457196626896148),
  'quant_VaR': np.float64(5.471559957029952)}]

In [50]:
meanrets = [x['meanret'] for x in best_model]
VaRs = [max(x['GARCH_VaR'], x['quant_VaR']) for x in best_model] # selecting the most conservative VaR estimate for each asset

print(VaRs)

VaR_constraint = 5 # since returns are expressed in %, the VaR constraint is also expressed in %
max_weight = np.array([min(1, VaR_constraint / var) for var in VaRs]) # because assets are assumed to be uncorrelated, we can simply consider portfolios that contain a single asset maximising its VaR allocation and the rest in cash
max_return = max_weight * meanrets
weightdict = {stocklist[i]:max_weight[i] for i in range(len(stocklist))}
retdict = {stocklist[i]:max_return[i] for i in range(len(stocklist))}
print(f"The maximum allocation that satisfies the VaR constraint for each asset is {weightdict}")
print(f"The returns that one would obtain from the maximum allocation are {retdict}")

[np.float64(3.2665112952789057), np.float64(14.892623315471734), np.float64(5.471559957029952)]
The maximum allocation that satisfies the VaR constraint for each asset is {'JNJ': np.float64(1.0), 'BTC-USD': np.float64(0.33573668614887825), 'AAPL': np.float64(0.913816176605342)}
The returns that one would obtain from the maximum allocation are {'JNJ': np.float64(0.11340282069253148), 'BTC-USD': np.float64(0.3718738261961301), 'AAPL': np.float64(0.41135424227173295)}


#### Hence, the optimal allocation is 91.38% funds to AAPL and 9% kept in cash, for a 1-week VaR of 4.99% and an expected 1-week return of 0.41%